In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#needs to be 3 dimensional. theta is a np.array of 3 rotation angles correspoinding to the x,y and z axis
#returns rotation matrix for a given angle
def rotation_mat(theta):
   
    R_x = np.asmatrix([[1,0,0],[0,np.cos(theta[0]),-np.sin(theta[0])],[0,np.sin(theta[0]),np.cos(theta[0])]])
    R_y = np.asmatrix([[np.cos(theta[1]),0,np.sin(theta[1])],[0,1,0],[-np.sin(theta[1]),0,np.cos(theta[1])]])
    R_z = np.asmatrix([[np.cos(theta[2]),-np.sin(theta[2]),0],[np.sin(theta[2]),np.cos(theta[2]),0],[0,0,1]])
    
    #create rotation matrix
    R = R_z @ R_y @ R_x
    
    return R


def gradient_theta(R,coords):
    #create skew symmetric matrix
   
    x = coords[0] 
    y = coords[1]
    z = coords[2]
    
    S = np.asmatrix([[0,-z,y],[z,0,-x],[-y,x,0]])
    
    grad_theta = S @ R
    return grad_theta


def gradient_descent(coords_forecast,coords_train,theta_init,n_iter=5000,alpha=.0001):
    
    if coords_forecast.shape[0] != coords_train.shape[0]:
        print('Please make sure the data from the second cycle is the same size as the data from the first cycle')
        return
    
    
    L = np.zeros((n_iter))
    theta = np.zeros((theta_init.shape[0],n_iter))
    Loss = np.zeros((n_iter))
    #add first theta
    theta[:,0] = theta_init
   
     
    #compute gradient:
    for i in range(0,(n_iter-1)):
        R = rotation_mat(theta[:,i])
        #rescale vectors
        
        
        #caclculate loss function
        Loss[i] = np.mean(np.array(([.5*np.linalg.norm(coords_forecast[k,:].T - R @ coords_train[k,:].T)**2 for k in range(0,coords_train.shape[0])])))
        
        if i > 1:
            if abs(Loss[i] - Loss[(i-1)]) < 1e4:
                   
                return theta,Loss,L[i]
        #grad_theta = gradient_theta(R,coords_train[k,:].T)
        
        L[i] = np.mean(np.array([-(coords_forecast[k,:].T/np.linalg.norm(coords_forecast[k,:].T) - R @ coords_train[k,:].T/np.linalg.norm(coords_train[k,:].T) @ gradient_theta(R,coords_train[k,:].T//np.linalg.norm(coords_train[k,:].T))) for k in range(0, coords_train.shape[0])]))
        
        #print(-(coords_forecast[i,:].T - R @ coords_train[i,:].T) @ gradient_theta(R,coords_train[i,:].T))
        
        #update theta
        theta[:,(i+1)] = (theta[:,i] - alpha * L[i])%2*np.pi 
        
        
    
        if i % 250 == 0:
            print(i) 
     
    
    return  theta,Loss,L[i]

In [ ]:
venus = pd.read_csv("Venus2x.csv")
length = venus.shape[0]
#first cycles data
dist_train = venus['Distance'][:(int(length/2))]
train_asc = venus['Right Ascension'][:(int(length/2))]
train_dec = venus['Declination'][:(int(length/2))]

#second cycles data
dist_test = venus['Distance'][(int(length/2)):]
test_asc = venus['Right Ascension'][(int(length/2)):]
test_dec = venus['Declination'][(int(length/2)):]

x1, y1, z1 = convert_cartesian(np.radians(train_asc), np.radians(train_dec), np.array(dist_train))
x2, y2, z2 = convert_cartesian(np.radians(test_asc), np.radians(test_dec), np.array(dist_train))

train = np.column_stack((x1,y1,z1))
test = np.column_stack((x2,y2,z2))

theta_init = np.random.uniform(0,2*np.pi,3)

opt_theta,loss,l = gradient_descent(test,train,theta_init,alpha = .0001)

Grid search attempt


In [ ]:

#grid search attempt

n = 25

theta_1 = np.linspace(0,2*np.pi,n)
theta_2 = np.linspace(0,2*np.pi,n)
theta_3 = np.linspace(0,2*np.pi,n)

train = np.column_stack((x1,y1,z1))
test = np.column_stack((x2,y2,z2))

Loss = np.zeros((n,n,n))
z = 0
for i in range(0,n):
    for j in range(0,n):
        for k in range(0,n):
            theta = (theta_1[i],theta_2[j],theta_3[k])
            R = rotation_mat(theta)
            
            Loss[i,j,k] = np.mean(np.array(([.5*np.linalg.norm(test[k,:].T - R @ train[k,:].T)**2 for k in range(0,train.shape[0])])))
            if z % 1000 == 0 :
               print(z)
            z +=1

Tried to narrow down where we got the lowest score, however it seems that there are alot of local mins.

In [ ]:
n = 15

theta_1 = np.linspace(2.9,3.5,n)
theta_2 = np.linspace(3.1,3.7*np.pi,n)
theta_3 = np.linspace(3.75,4.1,n)

train = np.column_stack((x1,y1,z1))
test = np.column_stack((x2,y2,z2))

Loss = np.zeros((n,n,n))
z = 0
for i in range(0,n):
    for j in range(0,n):
        for k in range(0,n):
            theta = (theta_1[i],theta_2[j],theta_3[k])
            R = rotation_mat(theta)
            
            Loss[i,j,k] = np.mean(np.array(([.5*np.linalg.norm(test[k,:].T - R @ train[k,:].T)**2 for k in range(0,train.shape[0])])))
            if z % 1000 == 0 :
                print(z)
            z +=1